# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
# Convert Max Temp from Kelvin to Celsius
city_data_df["Max Temp (C)"] = city_data_df["Max Temp"] - 273.15

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp (C)
0,0,edinburgh of the seven seas,-37.0676,-12.3116,288.59,88,100,17.24,SH,1700891870,15.44
1,1,college,64.8569,-147.8028,264.12,84,75,0.00,US,1700891870,-9.03
2,2,bethel,41.3712,-73.4140,272.24,61,0,1.54,US,1700891870,-0.91
3,3,udachny,66.4167,112.4000,246.73,75,100,1.99,RU,1700891870,-26.42
4,4,vila velha,-20.3297,-40.2925,296.12,77,40,9.26,BR,1700891871,22.97


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot_1
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles="OSM"
)

# Display the map plot_1
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

In [4]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    c='Humidity',
    hover_cols=['City', 'Humidity'],
    geo=True,
    tiles="OSM",
    size='Humidity',
    title='City Map with Humidity',
    frame_width=800,
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp (C)"] > 21) & (city_data_df["Max Temp (C)"] < 27) &
    (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp (C)
338,338,lazaro cardenas,17.9583,-102.2000,296.75,81,0,1.77,MX,1700891964,23.60
341,341,coahuayana de hidalgo,18.7000,-103.6583,295.09,72,0,1.99,MX,1700891965,21.94
471,471,khajuraho group of monuments,24.8500,79.9333,298.85,27,0,1.64,IN,1700892002,25.70
528,528,baiti,13.7450,-14.7482,294.37,20,0,1.12,SN,1700892046,21.22
534,534,toliara,-23.3500,43.6667,299.90,66,0,2.04,MG,1700892047,26.75


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df
hotel_df = ideal_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp (C),Hotel Name
338,338,lazaro cardenas,17.9583,-102.2000,296.75,81,0,1.77,MX,1700891964,23.60,
341,341,coahuayana de hidalgo,18.7000,-103.6583,295.09,72,0,1.99,MX,1700891965,21.94,
471,471,khajuraho group of monuments,24.8500,79.9333,298.85,27,0,1.64,IN,1700892002,25.70,
528,528,baiti,13.7450,-14.7482,294.37,20,0,1.12,SN,1700892046,21.22,
534,534,toliara,-23.3500,43.6667,299.90,66,0,2.04,MG,1700892047,26.75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000  # You can adjust the radius as needed
params = {
    "type": "lodging",
    "radius": radius,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"point:{lng},{lat}"
    params["bias"] = f"point:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
lazaro cardenas - nearest hotel: No hotel found
coahuayana de hidalgo - nearest hotel: No hotel found
khajuraho group of monuments - nearest hotel: No hotel found
baiti - nearest hotel: No hotel found
toliara - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp (C),Hotel Name
338,338,lazaro cardenas,17.9583,-102.2000,296.75,81,0,1.77,MX,1700891964,23.60,No hotel found
341,341,coahuayana de hidalgo,18.7000,-103.6583,295.09,72,0,1.99,MX,1700891965,21.94,No hotel found
471,471,khajuraho group of monuments,24.8500,79.9333,298.85,27,0,1.64,IN,1700892002,25.70,No hotel found
528,528,baiti,13.7450,-14.7482,294.37,20,0,1.12,SN,1700892046,21.22,No hotel found
534,534,toliara,-23.3500,43.6667,299.90,66,0,2.04,MG,1700892047,26.75,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
# Configure the map plot_2
map_plot_2 = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    c="Humidity",
    hover_cols=["Humidity", "City", "Hotel Name", "Country"],
    geo=True,
    tiles="OSM"
)

# Display the map plot_2
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)